In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/home/codetrade/Downloads/CSV/Musical_instruments_reviews.csv',usecols=['reviewText','overall'])
data.head()

,reviewText,overall
0,"Not much to write about here, but it does exac...",5.0
1,The product does exactly as it should and is q...,5.0
2,The primary job of this device is to block the...,5.0
3,Nice windscreen protects my MXL mic and preven...,5.0
4,This pop filter is great. It looks and perform...,5.0


In [3]:
data['overall'].value_counts()

5.0    6938
4.0    2084
3.0     772
2.0     250
1.0     217
Name: overall, dtype: int64

In [4]:
# pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

In [5]:
# pip install textblob
# !python -m spacy download en
# !python -m spacy download en_core_web_sm


In [6]:
import preprocess_kgptalkie as ps

2023-06-06 12:52:36.017060: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [7]:
import re

In [8]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [9]:
data['reviewText']=data['reviewText'].apply(lambda x: get_clean(x))

/home/codetrade/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [10]:
data.head()

,reviewText,overall
0,not much to write about here but it does exact...,5.0
1,the product does exactly as it should and is q...,5.0
2,the primary job of this device is to block the...,5.0
3,nice windscreen protects my mxl mic and preven...,5.0
4,this pop filter is great it looks and performs...,5.0


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


In [14]:
tfid=TfidfVectorizer(max_features=20000,ngram_range=(1,3),analyzer='char')

In [15]:
X =tfid.fit_transform(data['reviewText'])
y = data['overall']

In [16]:
X.shape,y.shape

((10261, 13500), (10261,))

In [17]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=0) 

In [18]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((8208, 13500), (2053, 13500), (8208,), (2053,))

In [19]:
clf=LinearSVC()
clf.fit(X_train,y_train)

LinearSVC()

In [20]:
y_pred=clf.predict(X_test)

In [21]:
print(y_pred)

[5. 3. 5. ... 5. 5. 5.]


In [23]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         1.0       0.33      0.03      0.05        39
         2.0       0.00      0.00      0.00        55
         3.0       0.35      0.06      0.10       134
         4.0       0.38      0.14      0.20       451
         5.0       0.71      0.96      0.81      1374

    accuracy                           0.68      2053
   macro avg       0.35      0.24      0.23      2053
weighted avg       0.59      0.68      0.60      2053

